In [ ]:
!pip install fuzzywuzzy[speedup]
!pip install googletrans==4.0.0-rc1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=70d53db884c34721e5c57045a8bafc4fe59a722406cd993961b5fe4326fc7085
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from googletrans import Translator
import re

# Load the data from the provided file paths
new_ocr_result_path = '/content/Ocr500.xlsx'
physician_data_path = '/content/Extracted_Physician_Data.xlsx'
institute_data_path = '/content/Extracted_Institute_Data.xlsx'

new_ocr_result_df = pd.read_excel(new_ocr_result_path)
physician_data_df = pd.read_excel(physician_data_path)
institute_data_df = pd.read_excel(institute_data_path)

translator = Translator()

# Function to translate Bengali text to English
def translate_text(text):
    try:
        translated = translator.translate(text, src='bn', dest='en')
        return translated.text
    except Exception as e:
        return ""

# Preprocess text for matching
def preprocess_text(text):
    text = text.replace('DR.', 'DR_')  # Temporarily replace 'DR.' to retain it
    text = re.sub(r'\.', '', text)  # Remove all other periods
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove non-alphabetic characters
    text = text.replace('DR_', 'DR.')  # Restore 'DR.'
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.strip().upper()  # Convert to uppercase

# Preprocess text for exact matching
def preprocess_text_exact(text):
    text = re.sub(r'\.', '', text)  # Remove all periods
    text = re.sub(r'\s+', '', text)  # Remove all spaces
    text = re.sub(r'[^A-Za-z]', '', text)  # Remove non-alphabetic characters
    return text.strip().upper()  # Convert to uppercase

# New function to process an image using Code3
def process_image_code3(extracted_text, phy_df):
    # Separate Bengali and English text
    bangla_text = ''.join([char for char in extracted_text if '\u0980' <= char <= '\u09FF'])
    english_text = ''.join([char for char in extracted_text if char not in bangla_text])

    # Translate Bengali text to English
    translated_text = translate_text(bangla_text)

    # Remove special characters and spaces
    english_text = re.sub(r'[^A-Za-z\s]', '', english_text).replace(' ', '')
    translated_text = re.sub(r'[^A-Za-z\s]', '', translated_text).replace(' ', '')

    # Merge texts
    merged_text = (english_text + translated_text).upper()
    print(f"Preprocessed Merged Text (Code3): {merged_text}")

    # Use fuzzy matching to find the best match for each physician name
    best_match = None
    best_score = 0
    for index, row in phy_df.iterrows():
        phy_nm = preprocess_text_exact(row['PHY_NM'].upper())
        phy_id = row['PHY_ID']
        match_score = fuzz.partial_ratio(phy_nm, merged_text)
        if match_score > best_score:  # Find the best match
            best_score = match_score
            best_match = (row['PHY_NM'], phy_id)

    if best_score > 80:  # Threshold for fuzzy matching
        print(f"Matched Doctor Name (Code3): {best_match[0]}, ID: {best_match[1]}, Score: {best_score}\n")
        return best_match[0], best_match[1]
    else:
        return None, None

# Function to process an image using Code1
def process_image_code1(merged_string, phy_df):
    preprocessed_merged_string = preprocess_text(merged_string)
    print("preprocessed_merged_string (Code1) =", preprocessed_merged_string)

    # Use fuzzy matching to find the best match for each physician name
    best_match = None
    best_score = 0
    for index, row in phy_df.iterrows():
        phy_nm = preprocess_text(row['PHY_NM'].upper())
        phy_id = row['PHY_ID']
        match_score = fuzz.partial_ratio(phy_nm, preprocessed_merged_string)
        if match_score > best_score:  # Find the best match
            best_score = match_score
            best_match = (row['PHY_NM'], phy_id)

    if best_score > 80:  # Threshold for fuzzy matching
        print(f"Matched Doctor Name (Code1): {best_match[0]}, ID: {best_match[1]}, Score: {best_score}\n")
        return best_match[0], best_match[1]
    else:
        return None, None

# Function to process an image using Code2
def process_image_code2(merged_string, phy_df):
    preprocessed_merged_string = preprocess_text_exact(merged_string)
    print("preprocessed_merged_string (Code2) =", preprocessed_merged_string)

    # Check for exact matches in the physician names
    best_match = None
    best_score = 0
    for index, row in phy_df.iterrows():
        phy_nm = preprocess_text_exact(row['PHY_NM'].upper())
        phy_id = row['PHY_ID']
        if phy_nm in preprocessed_merged_string:
            match_score = fuzz.partial_ratio(phy_nm, preprocessed_merged_string)
            if match_score > best_score:  # Find the best match
                best_score = match_score
                best_match = (row['PHY_NM'], phy_id)

    if best_score > 80:  # Threshold for fuzzy matching
        print(f"Matched Doctor Name (Code2): {best_match[0]}, ID: {best_match[1]}, Score: {best_score}\n")
        return best_match[0], best_match[1]
    else:
        return None, None

# Function to extract Institution_NM and Institution_ID using fuzzy matching
def get_institution_info(text, institute_data):
    best_match = None
    best_score = 0
    for inst_nm in institute_data['INS_NM1']:
        score = fuzz.partial_ratio(inst_nm.upper(), text)
        if score > best_score:
            best_score = score
            best_match = inst_nm
    if best_score > 80:  # Threshold for fuzzy matching
        inst_id = institute_data.loc[institute_data['INS_NM1'] == best_match, 'INSTCD'].values[0]
        return best_match, inst_id
    return None, None

# Function to extract date from text (both Bengali and English)
def extract_date(text):
    # English date patterns
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}', r'\d{2}-\d{2}-\d{4}', r'\d{2}/\d{2}/\d{4}', r'\d{4}/\d{2}/\d{2}',
        r'\d{4}\.\d{2}\.\d{2}', r'\d{2}\.\d{2}\.\d{4}'
    ]
    # Bengali date patterns (translated to their Unicode equivalents)
    bangla_digits = '০১২৩৪৫৬৭৮৯'
    date_patterns += [
        r'[{}]{{4}}-[{}]{{2}}-[{}]{{2}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{2}}-[{}]{{2}}-[{}]{{4}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{2}}/[{}]{{2}}/[{}]{{4}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{4}}/[{}]{{2}}/[{}]{{2}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{4}}\.[{}]{{2}}\.[{}]{{2}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{2}}\.[{}]{{2}}\.[{}]{{4}}'.format(bangla_digits, bangla_digits, bangla_digits)
    ]

    for pattern in date_patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(0)
    return None

# Reduce the number of entries for demonstration purposes
reduced_ocr_result_df = new_ocr_result_df.head(20)  # Select only the first 20 entries for demonstration

# Initialize an empty list to store the results
new_data_demo = []

# Initialize the results list
results = []

for index, row in reduced_ocr_result_df.iterrows():
    image_name = row['Image Name']
    extracted_text = row['Extracted Text']
    type_info = row['Type']
    translated_text = translate_text(extracted_text)
    merged_text = extracted_text.upper() + " " + translated_text.upper()

    # Print the preprocessed merged text
    print(f"Preprocessed Merged Text: {merged_text}")

    # Attempt matching using Code3 first
    phy_nm, phy_id = process_image_code3(extracted_text, physician_data_df)

    if not phy_nm:
        # If no match found using Code3, attempt
     if not phy_nm:
        # If no match found using Code3, attempt matching using Code2
        phy_nm, phy_id = process_image_code2(merged_text, physician_data_df)

        if not phy_nm:
            # If no match found using Code2, attempt matching using Code1
            phy_nm, phy_id = process_image_code1(merged_text, physician_data_df)

            if not phy_nm:
                print("All codes show No match found\n")

    inst_nm, inst_id = get_institution_info(merged_text, institute_data_df)
    date_info = extract_date(extracted_text)  # Search for dates directly in the main "Extracted Text"

    new_data_demo.append([
        image_name,
        extracted_text,
        phy_nm,
        phy_id,
        inst_nm,
        inst_id,
        type_info,
        date_info
    ])

# Convert the list to a DataFrame
new_df_demo = pd.DataFrame(new_data_demo, columns=[
    "Image Name",
    "Extracted Text",
    "PHY_NM from OCR",
    "PHY_ID",
    "Institution_NM",
    "Institution_ID",
    "Type",
    "Date"
])

# Display the new DataFrame
print(new_df_demo.head())

# Save the new DataFrame to an Excel file
output_path_demo = '/content/Processed_OCR_Data_Demo.xlsx'
new_df_demo.to_excel(output_path_demo, index=False)

# Provide a link to download the file
from google.colab import files
files.download(output_path_demo)

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from googletrans import Translator
import re

# Load the data from the provided file paths
new_ocr_result_path = '/content/Ocr500.xlsx'
physician_data_path = '/content/Extracted_PHY_NM_PHY_ID.xlsx'
institute_data_path = '/content/Institution DB (INSTCD and INS_NM1).xlsx'

new_ocr_result_df = pd.read_excel(new_ocr_result_path)
physician_data_df = pd.read_excel(physician_data_path)
institute_data_df = pd.read_excel(institute_data_path)

translator = Translator()

# Function to translate Bengali text to English
def translate_text(text):
    try:
        translated = translator.translate(text, src='bn', dest='en')
        return translated.text
    except Exception as e:
        return ""

# Preprocess text for matching
def preprocess_text(text):
    text = text.replace('DR.', 'DR_')  # Temporarily replace 'DR.' to retain it
    text = re.sub(r'\.', '', text)  # Remove all other periods
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove non-alphabetic characters
    text = text.replace('DR_', 'DR.')  # Restore 'DR.'
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.strip().upper()  # Convert to uppercase

# Preprocess text for exact matching
def preprocess_text_exact(text):
    text = re.sub(r'\.', '', text)  # Remove all periods
    text = re.sub(r'\s+', '', text)  # Remove all spaces
    text = re.sub(r'[^A-Za-z]', '', text)  # Remove non-alphabetic characters
    return text.strip().upper()  # Convert to uppercase

# New function to process an image using Code3
def process_image_code3(extracted_text, phy_df):
    # Separate Bengali and English text
    bangla_text = ''.join([char for char in extracted_text if '\u0980' <= char <= '\u09FF'])
    english_text = ''.join([char for char in extracted_text if char not in bangla_text])

    # Translate Bengali text to English
    translated_text = translate_text(bangla_text)

    # Remove special characters and spaces
    english_text = re.sub(r'[^A-Za-z\s]', '', english_text).replace(' ', '')
    translated_text = re.sub(r'[^A-Za-z\s]', '', translated_text).replace(' ', '')

    # Merge texts
    merged_text = (english_text + translated_text).upper()
    print(f"Preprocessed Merged Text (Code3): {merged_text}")

    # Use fuzzy matching to find the best match for each physician name
    best_match = None
    best_score = 0
    for index, row in phy_df.iterrows():
        phy_nm = preprocess_text_exact(row['PHY_NM'].upper())
        phy_id = row['PHY_ID']
        match_score = fuzz.partial_ratio(phy_nm, merged_text)
        if match_score > best_score:  # Find the best match
            best_score = match_score
            best_match = (row['PHY_NM'], phy_id)

    if best_score > 80:  # Threshold for fuzzy matching
        print(f"Matched Doctor Name (Code3): {best_match[0]}, ID: {best_match[1]}, Score: {best_score}\n")
        return best_match[0], best_match[1]
    else:
        return None, None

# Function to process an image using Code1
def process_image_code1(merged_string, phy_df):
    preprocessed_merged_string = preprocess_text(merged_string)
    print("preprocessed_merged_string (Code1) =", preprocessed_merged_string)

    # Use fuzzy matching to find the best match for each physician name
    best_match = None
    best_score = 0
    for index, row in phy_df.iterrows():
        phy_nm = preprocess_text(row['PHY_NM'].upper())
        phy_id = row['PHY_ID']
        match_score = fuzz.partial_ratio(phy_nm, preprocessed_merged_string)
        if match_score > best_score:  # Find the best match
            best_score = match_score
            best_match = (row['PHY_NM'], phy_id)

    if best_score > 80:  # Threshold for fuzzy matching
        print(f"Matched Doctor Name (Code1): {best_match[0]}, ID: {best_match[1]}, Score: {best_score}\n")
        return best_match[0], best_match[1]
    else:
        return None, None

# Function to process an image using Code2
def process_image_code2(merged_string, phy_df):
    preprocessed_merged_string = preprocess_text_exact(merged_string)
    print("preprocessed_merged_string (Code2) =", preprocessed_merged_string)

    # Check for exact matches in the physician names
    best_match = None
    best_score = 0
    for index, row in phy_df.iterrows():
        phy_nm = preprocess_text_exact(row['PHY_NM'].upper())
        phy_id = row['PHY_ID']
        if phy_nm in preprocessed_merged_string:
            match_score = fuzz.partial_ratio(phy_nm, preprocessed_merged_string)
            if match_score > best_score:  # Find the best match
                best_score = match_score
                best_match = (row['PHY_NM'], phy_id)

    if best_score > 80:  # Threshold for fuzzy matching
        print(f"Matched Doctor Name (Code2): {best_match[0]}, ID: {best_match[1]}, Score: {best_score}\n")
        return best_match[0], best_match[1]
    else:
        return None, None

# Function to extract Institution_NM and Institution_ID using fuzzy matching
def get_institution_info(text, institute_data):
    best_match = None
    best_score = 0
    for inst_nm in institute_data['INS_NM1']:
        score = fuzz.partial_ratio(inst_nm.upper(), text)
        if score > best_score:
            best_score = score
            best_match = inst_nm
    if best_score > 80:  # Threshold for fuzzy matching
        inst_id = institute_data.loc[institute_data['INS_NM1'] == best_match, 'INSTCD'].values[0]
        return best_match, inst_id
    return None, None

# Function to extract date from text (both Bengali and English)
def extract_date(text):
    # English date patterns
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}', r'\d{2}-\d{2}-\d{4}', r'\d{2}/\d{2}/\d{4}', r'\d{4}/\d{2}/\d{2}',
        r'\d{4}\.\d{2}\.\d{2}', r'\d{2}\.\d{2}\.\d{4}'
    ]
    # Bengali date patterns (translated to their Unicode equivalents)
    bangla_digits = '০১২৩৪৫৬৭৮৯'
    date_patterns += [
        r'[{}]{{4}}-[{}]{{2}}-[{}]{{2}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{2}}-[{}]{{2}}-[{}]{{4}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{2}}/[{}]{{2}}/[{}]{{4}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{4}}/[{}]{{2}}/[{}]{{2}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{4}}\.[{}]{{2}}\.[{}]{{2}}'.format(bangla_digits, bangla_digits, bangla_digits),
        r'[{}]{{2}}\.[{}]{{2}}\.[{}]{{4}}'.format(bangla_digits, bangla_digits, bangla_digits)
    ]

    for pattern in date_patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(0)
    return None

# Reduce the number of entries for demonstration purposes
reduced_ocr_result_df = new_ocr_result_df.head(522)  # Select only the first 20 entries for demonstration

# Initialize an empty list to store the results
new_data_demo = []

# Initialize the results list
results = []

for index, row in reduced_ocr_result_df.iterrows():
    image_name = row['Image Name']
    extracted_text = row['Extracted Text']
    type_info = row['Type']
    translated_text = translate_text(extracted_text)
    merged_text = extracted_text.upper() + " " + translated_text.upper()

    # Print the preprocessed merged text
    print(f"Preprocessed Merged Text: {merged_text}")

    # Attempt matching using Code3 first
    phy_nm, phy_id = process_image_code3(extracted_text, physician_data_df)
    if not phy_nm:
        # If no match found using Code3, attempt
     if not phy_nm:
        # If no match found using Code3, attempt matching using Code2
        phy_nm, phy_id = process_image_code2(merged_text, physician_data_df)

        if not phy_nm:
            # If no match found using Code2, attempt matching using Code1
            phy_nm, phy_id = process_image_code1(merged_text, physician_data_df)

            if not phy_nm:
                print("All codes show No match found\n")

    inst_nm, inst_id = get_institution_info(merged_text, institute_data_df)
    date_info = extract_date(extracted_text)  # Search for dates directly in the main "Extracted Text"

    new_data_demo.append([
        image_name,
        extracted_text,
        phy_nm,
        phy_id,
        inst_nm,
        inst_id,
        type_info,
        date_info
    ])

# Convert the list to a DataFrame
new_df_demo = pd.DataFrame(new_data_demo, columns=[
    "Image Name",
    "Extracted Text",
    "PHY_NM from OCR",
    "PHY_ID",
    "Institution_NM",
    "Institution_ID",
    "Type",
    "Date"
])

# Display the new DataFrame
print(new_df_demo.head())

# Save the new DataFrame to an Excel file
output_path_demo = '/content/Processed_OCR_Data_Demo.xlsx'
new_df_demo.to_excel(output_path_demo, index=False)

# Provide a link to download the file
from google.colab import files
files.download(output_path_demo)

Streaming output truncated to the last 5000 lines.
17893618/28
01724077
P
FOFARANT
16. DICLOFEN
WWW/F
22
(ENG)
50
CUP. FINIX 20
পদিন
दित
YOU, OSTOCALES
DICLOGEL
দৈনিক ৩
৪ ৰাৱ
-83/VB-8080, 35-
নিঃ-৫৮/২০২২-২৩, ২ কোটি রুপি
TM
SUPHA B
ЭРОТО
BARDD
2022-201
R
WEKEL DE HOSPITAL/ CO
CEFEN WWW
LATTER
B
HAND
PAIN
OUTPATIENT
17893618/28
01724077
P
FOFARANT
16. DICLOFEN
WWW/F
22
(ENG)
50
CUP.FINIX 20
VEIN
DISRUPTED
Y, OSTOCALES
DICLOGEL
DAILY 1
3
-83/VB-8080, 35-
UNTI-1/2022-21, RS 2 CRORE
TM
SUPHA B
ЭРОТО
BARDD
2022-201
R
WEKEL DE
Preprocessed Merged Text (Code3): 
CEFENWWW

B
HAND
PAIN



P
FOFARANT
DICLOFEN
WWWF

ENG

CUPFINIX


YOUOSTOCALES
DICLOGEL


VB

TM
SUPHAB

BARDD

R
WEKELDEHOSPITALDIVINITYPATROTOPITPADDINIKDAUNICROUP
preprocessed_merged_string (Code2) = CEFENWWWBHANDPAINPFOFARANTDICLOFENWWWFENGCUPFINIXYOUOSTOCALESDICLOGELVBTMSUPHABBARDDRWEKELDEHOSPITALCOCEFENWWWLATTERBHANDPAINOUTPATIENTPFOFARANTDICLOFENWWWFENGCUPFINIXVEINDISRUPTEDYOSTOCALESDICLOGELDAILYVBUNTIRSCRORETMSUPHABBARDDRWEKEL

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re

# Function to find dates in a given text
def find_dates(text):
    english_date_pattern = r'\b(?:\d{1,2}[-/th|st|nd|rd]\s?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s?\d{2,4})\b|\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4})\b|\b(?:\d{4}[-/]\d{1,2}[-/]\d{1,2})\b'
    bengali_date_pattern = r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4})\b'

    english_dates = re.findall(english_date_pattern, text)
    bengali_dates = re.findall(bengali_date_pattern, text)

    return english_dates, bengali_dates

# Load the data from the specified file path
file_path = '/content/Ocr500.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# List to store the results
results = []

# Loop through each row in the data
for index, row in data.iterrows():
    image_name = row['Image Name']
    extracted_text = row['Extracted Text']
    english_dates, bengali_dates = find_dates(extracted_text)
    if english_dates or bengali_dates:
        results.append({
            'Image Name': image_name,
            'Extracted Text': extracted_text,
            'English Dates': english_dates,
            'Bengali Dates': bengali_dates
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
output_file_path = '/content/extracted_dates.xlsx'
results_df.to_excel(output_file_path, index=False)

# Download the file
from google.colab import files
files.download(output_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re

# Function to find dates in a given text
def find_dates(text):
    english_date_pattern = r'\b(?:0?[1-9]|[12][0-9]|3[01])[-/th|st|nd|rd\s]?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s?(?:200\d|201\d|202\d|203\d|204\d|2050)\b|\b(?:0?[1-9]|[12][0-9]|3[01])[-/](0?[1-9]|1[012])[-/](?:200\d|201\d|202\d|203\d|204\d|2050)\b|\b(?:200\d|201\d|202\d|203\d|204\d|2050)[-/](0?[1-9]|1[012])[-/](0?[1-9]|[12][0-9]|3[01])\b'
    bengali_date_pattern = r'\b(?:[১-৯]|১[০-৯]|২[০-৯]|৩০|৩১)[-/](?:[১-৯]|১[০-২])[-/](?:২০০[০-৯]|২০১[০-৯]|২০২[০-৯]|২০৩[০-৯]|২০৪[০-৯]|২০৫০)\b'

    english_dates = re.findall(english_date_pattern, text)
    bengali_dates = re.findall(bengali_date_pattern, text)

    return english_dates, bengali_dates

# Load the data from the specified file path
file_path = '/content/Ocr500.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# List to store the results
results = []

# Loop through each row in the data
for index, row in data.iterrows():
    image_name = row['Image Name']
    extracted_text = row['Extracted Text']
    english_dates, bengali_dates = find_dates(extracted_text)
    if english_dates or bengali_dates:
        results.append({
            'Image Name': image_name,
            'Extracted Text': extracted_text,
            'English Dates': english_dates,
            'Bengali Dates': bengali_dates
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
output_file_path = '/content/extracted_dates.xlsx'
results_df.to_excel(output_file_path, index=False)

# Download the file
from google.colab import files
files.download(output_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re

# Function to find dates in a given text
def find_dates(text):
    # english_date_pattern = r'\b(?:0?[1-9]|[12][0-9]|3[01])[-/th|st|nd|rd\s]?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s?(?:200\d|201\d|202\d|203\d|204\d|2050)\b|\b(?:0?[1-9]|[12][0-9]|3[01])[-/](0?[1-9]|1[012])[-/](?:200\d|201\d|202\d|203\d|204\d|2050)\b|\b(?:200\d|201\d|202\d|203\d|204\d|2050)[-/](0?[1-9]|1[012])[-/](0?[1-9]|[12][0-9]|3[01])\b'
    # bengali_date_pattern = r'\b(?:[১-৯]|১[০-৯]|২[০-৯]|৩০|৩১)[-/](?:[১-৯]|১[০-২])[-/](?:২০০[০-৯]|২০১[০-৯]|২০২[০-৯]|২০৩[০-৯]|২০৪[০-৯]|২০৫০)\b'
    english_date_pattern = r'\b(?:\d{1,2}[-/th|st|nd|rd]\s?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s?\d{2,4})\b|\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4})\b|\b(?:\d{4}[-/]\d{1,2}[-/]\d{1,2})\b'
    bengali_date_pattern = r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4})\b'
    english_dates = re.findall(english_date_pattern, text)
    bengali_dates = re.findall(bengali_date_pattern, text)

    return english_dates + bengali_dates

# Load the data from the specified file path
file_path = '/content/Ocr500.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# List to store the results
results = []

# Loop through each row in the data
for index, row in data.iterrows():
    image_name = row['Image Name']
    extracted_text = row['Extracted Text']
    dates = find_dates(extracted_text)
    if dates:
        results.append({
            'Image Name': image_name,
            'Extracted Text': extracted_text,
            'Dates': dates
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
output_file_path = '/content/extracted_dates.xlsx'
results_df.to_excel(output_file_path, index=False)

# Download the file
from google.colab import files
files.download(output_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from googletrans import Translator
import re

# Load the data from the provided file paths
new_ocr_result_path = '/content/Ocr500.xlsx'
physician_data_path = '/content/Extracted_PHY_NM_PHY_ID.xlsx'
institute_data_path = '/content/Institution DB (INSTCD and INS_NM1).xlsx'

new_ocr_result_df = pd.read_excel(new_ocr_result_path)
physician_data_df = pd.read_excel(physician_data_path)
institute_data_df = pd.read_excel(institute_data_path)

translator = Translator()

# Function to translate Bengali text to English
def translate_text(text):
    try:
        translated = translator.translate(text, src='bn', dest='en')
        return translated.text
    except Exception as e:
        return ""

# Preprocess text for matching
def preprocess_text(text):
    text = text.replace('DR.', 'DR_')  # Temporarily replace 'DR.' to retain it
    text = re.sub(r'\.', '', text)  # Remove all other periods
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove non-alphabetic characters
    text = text.replace('DR_', 'DR.')  # Restore 'DR.'
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.strip().upper()  # Convert to uppercase

# Preprocess text for exact matching
def preprocess_text_exact(text):
    text = re.sub(r'\.', '', text)  # Remove all periods
    text = re.sub(r'\s+', '', text)  # Remove all spaces
    text = re.sub(r'[^A-Za-z]', '', text)  # Remove non-alphabetic characters
    return text.strip().upper()  # Convert to uppercase

# New function to process an image using Code3
def process_image_code3(extracted_text, phy_df):
    # Separate Bengali and English text
    bangla_text = ''.join([char for char in extracted_text if '\u0980' <= char <= '\u09FF'])
    english_text = ''.join([char for char in extracted_text if char not in bangla_text])

    # Translate Bengali text to English
    translated_text = translate_text(bangla_text)

    # Remove special characters and spaces
    english_text = re.sub(r'[^A-Za-z\s]', '', english_text).replace(' ', '')
    translated_text = re.sub(r'[^A-Za-z\s]', '', translated_text).replace(' ', '')

    # Merge texts
    merged_text = (english_text + translated_text).upper()
    print(f"Preprocessed Merged Text (Code3): {merged_text}")

    # Use fuzzy matching to find the best match for each physician name
    best_match = None
    best_score = 0
    for index, row in phy_df.iterrows():
        phy_nm = preprocess_text_exact(row['PHY_NM'].upper())
        phy_id = row['PHY_ID']
        match_score = fuzz.partial_ratio(phy_nm, merged_text)
        if match_score > best_score:  # Find the best match
            best_score = match_score
            best_match = (row['PHY_NM'], phy_id)

    if best_score > 80:  # Threshold for fuzzy matching
        print(f"Matched Doctor Name (Code3): {best_match[0]}, ID: {best_match[1]}, Score: {best_score}\n")
        return best_match[0], best_match[1]
    else:
        return None, None

# Function to process an image using Code1
def process_image_code1(merged_string, phy_df):
    preprocessed_merged_string = preprocess_text(merged_string)
    print("preprocessed_merged_string (Code1) =", preprocessed_merged_string)

    # Use fuzzy matching to find the best match for each physician name
    best_match = None
    best_score = 0
    for index, row in phy_df.iterrows():
        phy_nm = preprocess_text(row['PHY_NM'].upper())
        phy_id = row['PHY_ID']
        match_score = fuzz.partial_ratio(phy_nm, preprocessed_merged_string)
        if match_score > best_score:  # Find the best match
            best_score = match_score
            best_match = (row['PHY_NM'], phy_id)

    if best_score > 80:  # Threshold for fuzzy matching
        print(f"Matched Doctor Name (Code1): {best_match[0]}, ID: {best_match[1]}, Score: {best_score}\n")
        return best_match[0], best_match[1]
    else:
        return None, None

# Function to process an image using Code2
def process_image_code2(merged_string, phy_df):
    preprocessed_merged_string = preprocess_text_exact(merged_string)
    print("preprocessed_merged_string (Code2) =", preprocessed_merged_string)

    # Check for exact matches in the physician names
    best_match = None
    best_score = 0
    for index, row in phy_df.iterrows():
        phy_nm = preprocess_text_exact(row['PHY_NM'].upper())
        phy_id = row['PHY_ID']
        if phy_nm in preprocessed_merged_string:
            match_score = fuzz.partial_ratio(phy_nm, preprocessed_merged_string)
            if match_score > best_score:  # Find the best match
                best_score = match_score
                best_match = (row['PHY_NM'], phy_id)

    if best_score > 80:  # Threshold for fuzzy matching
        print(f"Matched Doctor Name (Code2): {best_match[0]}, ID: {best_match[1]}, Score: {best_score}\n")
        return best_match[0], best_match[1]
    else:
        return None, None

# Function to extract Institution_NM and Institution_ID using fuzzy matching
def get_institution_info(text, institute_data):
    best_match = None
    best_score = 0
    for inst_nm in institute_data['INS_NM1']:
        score = fuzz.partial_ratio(inst_nm.upper(), text)
        if score > best_score:
            best_score = score
            best_match = inst_nm
    if best_score > 80:  # Threshold for fuzzy matching
        inst_id = institute_data.loc[institute_data['INS_NM1'] == best_match, 'INSTCD'].values[0]
        return best_match, inst_id
    return None, None

# Function to find dates in text
def find_dates(text):
    english_date_pattern = r'\b(?:\d{1,2}[-/th|st|nd|rd]\s?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)|Jul(?:y)|Aug(?:ust)|Sep(?:tember)|Oct(?:ober)|Nov(?:ember)|Dec(?:ember)?)\s?\d{2,4})\b|\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4})\b|\b(?:\d{4}[-/]\d{1,2}[-/]\d{1,2})\b'
    bengali_date_pattern = r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4})\b'

    english_dates = re.findall(english_date_pattern, text)
    bengali_dates = re.findall(bengali_date_pattern, text)

    return english_dates, bengali_dates

# Reduce the number of entries for demonstration purposes
reduced_ocr_result_df = new_ocr_result_df.head(522)  # Select only the first 20 entries for demonstration

# Initialize an empty list to store the results
new_data_demo = []

for index, row in reduced_ocr_result_df.iterrows():
    image_name = row['Image Name']
    extracted_text = row['Extracted Text']
    type_info = row['Type']
    translated_text = translate_text(extracted_text)
    merged_text = extracted_text.upper() + " " + translated_text.upper()

    # Print the preprocessed merged text
    print(f"Preprocessed Merged Text: {merged_text}")

    # Attempt matching using Code3 first
    phy_nm, phy_id = process_image_code3(extracted_text, physician_data_df)

    if not phy_nm:
        # If no match found using Code3, attempt matching using Code2
        phy_nm, phy_id = process_image_code2(merged_text, physician_data_df)

        if not phy_nm:
            # If no match found using Code2, attempt matching using Code1
            phy_nm, phy_id = process_image_code1(merged_text, physician_data_df)

            if not phy_nm:
                print("All codes show No match found\n")

    inst_nm, inst_id = get_institution_info(merged_text, institute_data_df)

    # Find dates
    english_dates, bengali_dates = find_dates(extracted_text)
    all_dates = list(set(english_dates + bengali_dates))  # Combine and remove duplicates

    new_data_demo.append([
        image_name,
        extracted_text,
        phy_nm,
        phy_id,
        inst_nm,
        inst_id,
        type_info,
        ', '.join(all_dates)  # Combine all dates into a single string
    ])

# Convert the list to a DataFrame
new_df_demo = pd.DataFrame(new_data_demo, columns=[
    "Image Name",
    "Extracted Text",
    "PHY_NM from OCR",
    "PHY_ID",
    "Institution_NM",
    "Institution_ID",
    "Type",
    "Extracted Date"
])

# Display the new DataFrame
print(new_df_demo.head())

# Save the new DataFrame to an Excel file
output_path_demo = '/content/Processed_OCR_Data_Demo.xlsx'
new_df_demo.to_excel(output_path_demo, index=False)

# Provide a link to download the file
from google.colab import files
files.download(output_path_demo)


Streaming output truncated to the last 5000 lines.
TAB, MONTAIN O
04012 - ১৫ দিন
TAB, OPTOM- 20
27070
-
নং সম(বাঃবাঃকোঃ)/ভেটিং/ফ-৪১/৬৯-৪৩৪৫, তাং ১৯-৮-৮৯২৮-
বাঃ নিঃ মুঃ ৫৮/২০২২-২৩, ২ কোটি রুপি, মুদ্রণাদেশ নং-৩৩/২০২২-২৩। BANGLADESH FORM
200.
OUTPATIENT
200
HOSPITAL/ BAY
178657
DATE: 1
AGE ...
11/8/28
DISEASE
ROX TO DATE
RA
TREATMENT
1-10.P/C
1-
TAB.B126
1+1+1
6 MONTHS
TAB, MONTAIN O
04012 - 3 DAYS
TAB, OPTOM- 20
27070
-
NO. (BABACO)/VETEATING/F-1/1-5, TANG 1-3-12-
B: NON-MD. 1/222-21, RS 2 CRORE, PRINTING NO-1/2022-21.
Preprocessed Merged Text (Code3): 









ROX
RA

PC

TABB


TABMONTAINO

TABOPTOM



BANGLADESHIBHARATIDAMNITATTIKITAHASSIPATLABETRICHBHAGATHAKTARIKHAKTIKADISEASEMONTHDINANGSBAHBAHVOITINGTANGB
preprocessed_merged_string (Code2) = ROXRAPCTABBTABMONTAINOTABOPTOMBANGLADESHFORMOUTPATIENTHOSPITALBAYDATEAGEDISEASEROXTODATERATREATMENTPCTABBMONTHSTABMONTAINODAYSTABOPTOMNOBABACOVETEATINGFTANGBNONMDRSCROREPRINTINGNO
preprocessed_merged_string (Code1) = ROX RA PC TAB B TAB MONTAIN

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>